Example of simple network 

In [1]:
import torch
import torch.nn as nn

def create_neural_network(input_dims, output_dims, hidden_dims=[64, 32], activation=nn.ReLU()):
    layers = []
    
    # Add input layer
    layers.append(nn.Linear(input_dims, hidden_dims[0]))
    layers.append(activation)
    
    # Add hidden layers
    for i in range(1, len(hidden_dims)):
        layers.append(nn.Linear(hidden_dims[i-1], hidden_dims[i]))
        layers.append(activation)
    
    # Add output layer
    layers.append(nn.Linear(hidden_dims[-1], output_dims))
    
    # Create the neural network
    model = nn.Sequential(*layers)
    
    return model

# Example usage
input_dims = 10
output_dims = 5
hidden_dims = [64, 32, 16]  # You can adjust the number of hidden layers and their sizes
activation = nn.ReLU()

net = create_neural_network(input_dims, output_dims, hidden_dims, activation)
print(net)


Sequential(
  (0): Linear(in_features=10, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=16, bias=True)
  (5): ReLU()
  (6): Linear(in_features=16, out_features=5, bias=True)
)


Creating complex network with residual network

In [2]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.stride != 1 or identity.shape[1] != out.shape[1]:
            identity = nn.Conv2d(identity.shape[1], out.shape[1], kernel_size=1, stride=self.stride, bias=False)(identity)
            identity = nn.BatchNorm2d(out.shape[1])(identity)

        out += identity
        out = self.relu(out)

        return out

class ComplexNeuralNetwork(nn.Module):
    def __init__(self, input_dims, output_dims, conv_channels=[16, 32], hidden_dims=[64, 32], activation=nn.ReLU()):
        super(ComplexNeuralNetwork, self).__init__()
        
        layers = []
        
        # Initial convolutional layer
        layers.append(nn.Conv2d(in_channels=input_dims, out_channels=conv_channels[0], kernel_size=3, padding=1))
        layers.append(nn.BatchNorm2d(conv_channels[0]))
        layers.append(activation)
        
        # Convolutional layers with skip connections (using ResidualBlocks)
        in_channels = conv_channels[0]
        for out_channels in conv_channels:
            layers.append(ResidualBlock(in_channels, out_channels))
            in_channels = out_channels
        
        # Flatten the output of the convolutional layers
        layers.append(nn.Flatten())
        
        # Linear layers with skip connections
        in_features = in_channels
        for out_features in hidden_dims:
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.BatchNorm1d(out_features))
            layers.append(activation)
            in_features = out_features
        
        # Output linear layer
        layers.append(nn.Linear(in_features, output_dims))
        
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Example usage
input_dims = 1  # Number of input channels (e.g., for RGB images)
output_dims = 10  # Number of output classes
conv_channels = [16, 32]  # Number of channels in convolutional layers
hidden_dims = [64, 32]  # Sizes of hidden linear layers
activation = nn.ReLU()

net = ComplexNeuralNetwork(input_dims, output_dims, conv_channels, hidden_dims, activation).to("cuda")

print(net)


ComplexNeuralNetwork(
  (network): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): ResidualBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   

Downloading the training and validation dataset cifar10

In [6]:
from torchvision import datasets, transforms

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
train_dataset = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
test_dataset = datasets.MNIST('../data', train=False,
                   transform=transform)


In [ ]:
test_dataset.to("cuda")

In [7]:
next(iter(train_dataset))[0].shape

torch.Size([1, 28, 28])

Specifing the problem

In [ ]:
from evotorch.neuroevolution import SupervisedNE

mnist_problem = SupervisedNE(
    train_dataset,  # Using the dataset specified earlier
    net,  # Training the MNIST30K module designed earlier
    nn.CrossEntropyLoss(),  # Minimizing CrossEntropyLoss
    minibatch_size = 256,  # With a minibatch size of 256
    common_minibatch = True,  # Always using the same minibatch across all solutions on an actor
    num_actors = 4,  # The total number of CPUs used
    num_gpus_per_actor = 'max',  # Dividing all available GPUs between the 4 actors
    subbatch_size = 50,  # Evaluating solutions in sub-batches of size 50 ensures we won't run out of GPU memory for individual workers
)


[2023-08-31 23:42:59] INFO     <21732> evotorch.core: Instance of `SupervisedNE` (id:1977066052304) -- The `dtype` for the problem's decision variables is set as torch.float32
[2023-08-31 23:42:59] INFO     <21732> evotorch.core: Instance of `SupervisedNE` (id:1977066052304) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2023-08-31 23:42:59] INFO     <21732> evotorch.core: Instance of `SupervisedNE` (id:1977066052304) -- The `device` of the problem is set as cpu
[2023-08-31 23:42:59] INFO     <21732> evotorch.core: Instance of `SupervisedNE` (id:1977066052304) -- The number of actors that will be allocated for parallelized evaluation is 4


2023-08-31 23:43:03,782	INFO worker.py:1621 -- Started a local Ray instance.


In [ ]:
from evotorch.algorithms import SNES
searcher = SNES(mnist_problem, stdev_init = 1, popsize = 1000, distributed = True)


In [ ]:
from evotorch.logging import StdOutLogger, PandasLogger
stdout_logger = StdOutLogger(searcher, interval = 1)
pandas_logger = PandasLogger(searcher, interval = 1)


In [ ]:
searcher.run(10)


(EvaluationActor pid=31364) c:\Users\Asus\anaconda3\envs\torch-env\Lib\site-packages\evotorch\core.py:3425: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
(EvaluationActor pid=31364)   shares_storage = self._data.storage().data_ptr() == source._data.storage().data_ptr()


RayTaskError(RuntimeError): [36mray::EvaluationActor.call()[39m (pid=15768, ip=127.0.0.1, actor_id=ec842f8f74d2885b086432b201000000, repr=<evotorch.core.EvaluationActor object at 0x000001F4AF18C250>)
  File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
  File "c:\Users\Asus\anaconda3\envs\torch-env\Lib\site-packages\ray\_private\function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Asus\anaconda3\envs\torch-env\Lib\site-packages\ray\util\tracing\tracing_helper.py", line 464, in _resume_span
    return method(self, *_args, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Asus\anaconda3\envs\torch-env\Lib\site-packages\evotorch\core.py", line 185, in call
    return getattr(self._problem, method_name)(*args, **kwargs)
           ^^^^^^^^^^^^^^

2023-08-31 23:27:58,918	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::EvaluationActor.call() (pid=31376, ip=127.0.0.1, actor_id=c3fbdfbe427049d95e21120501000000, repr=<evotorch.core.EvaluationActor object at 0x000001E80F282090>)
  File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
  File "c:\Users\Asus\anaconda3\envs\torch-env\Lib\site-packages\ray\_private\function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Asus\anaconda3\envs\torch-env\Lib\site-packages\ray\util\tracing\tracing_helper.py", line 464, in _resume_span
    return method(self, *_args, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Asus\anaconda3\envs\torch-env\Lib\site-packages\evotorch\core.py", line 185, in call
    return geta